<a href="https://colab.research.google.com/github/sadnow/360Diffusion/blob/main/360Diffusion_AlphaTesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#360Diffusion - ESRGAN-Embedded Fast CLIP Guided Diffusion

---
**Latest Update: Alpha v1.2 - Ninja release 10/7/21**

- Complete overhaul of upscaling code, much easier to debug
- VRAM fixes, automatic performance_mode if upscale fails
- Debug mode now sets skip_timesteps to high #
- Nixed double-upscaler (may come back)


**Note: Default settings have been tested on an A100 with the `512 model` only, with `2048x2048`. None of the other settings have been tested.

For VRAM errors you can either raise `cutn_batches` or lower `cutn`. For P100, try `cutn` = 30 and `cutn_batches` = 4.

---
This release was not intended for a broad audience. Likely because it's either unfinished or needs more testing. If you've stubled upon this by accident, congrats!

---
*Expand this cell to see changelogs and credits-*

**UI simplifications and Real-ESRGAN integration originally added by sadnow (@sadly_existent) on 9/27/21. Latest revision pushed on 10/7/21. Updates can be found at https://github.com/sadnow/ESRGAN-UltraFast-CLIP-Guided-Diffusion-Colab if available.**

- Added form support for init images (supports both URL and filepath) and image_prompt
- Added A100 support (quite slow currently, patch advice credit goes to sportracer48)
- preemptive error correction for user input values 
- Documentation additions
- Notebook now does 256x diffusion and then upscales to 1024x1024 using Real-ESRGAN
- Upscalig uses gfpgan for face enhancement
- Added a "project name" setting and changed temp folder location
- Added a global scaling slider for simplicity
---
- Added 2048x2048 upscaling (double-upscaling)
  - Saves as PNG on first upscale now (*hotfix*)
- Added debug_mode (10/3)
- Added support for ESRNET and og ESRGAN models (10/3)



**I highly recommend trying out a silhouette as an init_image ^^**

<blockquote class="twitter-tweet"><p lang="en" dir="ltr">One of the best Panda generators out there <a href="https://t.co/H3Y3Im3vDZ">pic.twitter.com/H3Y3Im3vDZ</a></p>&mdash; Pandora&#39;s Box (@sadly_existent) <a href="https://twitter.com/sadly_existent/status/1444175519105310721?ref_src=twsrc%5Etfw">October 2, 2021</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

---

###This is a ninja release, meaning it was not intended for a broad audience. Likely because it's either unfinished or needs more testing. If you've stubled upon this by accident, congrats! 

## Credits
Msg me if you'd like your handles in here!

---

Original Notebook Katherine Crowson (https://github.com/crowsonkb, https://twitter.com/RiversHaveWings). It uses either OpenAI's 256x256 unconditional ImageNet or Katherine Crowson's fine-tuned 512x512 diffusion model (https://github.com/openai/guided-diffusion), together with CLIP (https://github.com/openai/CLIP) to connect text prompts with images. Katherine's original notebook can be found here: https://colab.research.google.com/drive/1QBsaDAZv8np29FPbvjffbE1eytoJcsgA

Modified by Daniel Russell (https://github.com/russelldc, https://twitter.com/danielrussruss) to include (hopefully) optimal params for quick generations in 15-100 timesteps rather than 1000, as well as more robust augmentations.

Further improvements from Dango233 and nsheppard helped improve the quality of diffusion in general, and especially so for shorter runs like this notebook aims to achieve. **Sep 19th 2021**

---

# GPU, Path & Model Settings

In [ ]:
#@markdown ##Check GPU
#@markdown ***This Notebook supports "Run All. Once everything is set to your liking, you can do `Ctrl+F9`!***
import torch
# Check the GPU status
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

a100_support = False #@param {type:"boolean"}

enable_error_checking = False #saves ram
if enable_error_checking:
  !nvidia-smi
else:
  !nvidia-smi
  !nvidia-smi -i 0 -e 0
#@markdown *Special thanks to sportracer48 and his Discord channel. If you want to make AI animations, he has the meats in closed beta.* https://www.patreon.com/sportsracer48
if a100_support:
  print("If you have an A100: Turn your cutn to 64-96 and keep your batch_n low (1 or 2), and it will be fast!")
  !pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html
  #!conda install pytorch torchvision torchaudio cudatoolkit=11.1 -c pytorch -c nvidia

# function runAllCells() {
#   const F9Event = {key: "F9", code: "F9", metaKey: true, keyCode: 120};
#   document.dispatchEvent(new KeyboardEvent("keydown", F9Event));
# } #https://stackoverflow.com/questions/65984431/run-all-cells-command-in-google-colab-programmatically

In [ ]:
def dir_make(a):
  import os.path
  from os import path
  if not path.exists(a):
    print("Creating"+a+"...")
    !mkdir -p $a

def dir_check(a):
  import os.path
  from os import path
  if not path.exists(a):
    return False
  else:
    return True

def file_check(a):
  import os.path
  from os import path
  if not path.isfile(a):
    return False
  else:
    return True

def image_upscale(model_path,scale,input,output):
    %cd /content/Real-ESRGAN/
    !python inference_realesrgan.py --model_path $model_path --netscale $scale --face_enhance --input $input --output $output --ext jpg
    %cd /content/

from google.colab import drive

#@title Choose model here:
diffusion_model = "512x512_diffusion_uncond_finetune_008100" #@param ["256x256_diffusion_uncond", "512x512_diffusion_uncond_finetune_008100"]

#@markdown `512 diffusion model` is new and has very little testing behind it. Play with `cutn` and `cutn_batches` to fix VRAM errors.
#diffusion_model = "256x256_diffusion_uncond"

#@markdown If you connect your Google Drive, you can save the final image of each run on your drive.

google_drive = True #@param {type:"boolean"}

#@markdown You can use your mounted Google Drive to load the model checkpoint file if you've already got a copy downloaded there. This will save time (and resources!) when you re-visit this notebook in the future.

#@markdown Click here if you'd like to save the diffusion model checkpoint file to (and/or load from) your Google Drive:
yes_please = True #@param {type:"boolean"}



_drive_location = '/content/drive/MyDrive/AI/Diffusion/' #@param{type:"string"}
contains_slash = (_drive_location.endswith('/'))
if not contains_slash:
  _drive_location = _drive_location + '/'


#####################################################################
##@title Google Drive & Download diffusion model

model_path = '/content/'
if google_drive:
    from google.colab import drive
    drive.mount('/content/drive')
    if yes_please:
        dir_make(_drive_location)
        model_path = _drive_location


if diffusion_model == '256x256_diffusion_uncond':
    !wget --continue 'https://openaipublic.blob.core.windows.net/diffusion/jul-2021/256x256_diffusion_uncond.pt' -P {model_path}
elif diffusion_model == '512x512_diffusion_uncond_finetune_008100':
    !wget --continue 'https://the-eye.eu/public/AI/models/512x512_diffusion_unconditional_ImageNet/512x512_diffusion_uncond_finetune_008100.pt' -P {model_path}
    #_diffusion_int = 512
if google_drive and not yes_please:
    model_path = _drive_location

#https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.1/ESRGAN_SRx4_DF2KOST_official-ff704c30.pth


# Install and import dependencies

In [ ]:
def install_ESRGAN():
    %cd /content/
    print("Installing libraries for Real-ESRGAN upscaling.")
    !git clone https://github.com/xinntao/Real-ESRGAN.git
    %cd /content/Real-ESRGAN
    !pip install basicsr -q
    !pip install facexlib -q
    !pip install gfpgan -q
    !pip install -r requirements.txt -q
    %cd /content/Real-ESRGAN
    !python setup.py develop -q
    !wget -nc https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models
    !wget -nc https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.2.4/RealESRGAN_x4plus_anime_6B.pth -P experiments/pretrained_models
    !wget -nc https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth -P experiments/pretrained_models
    !wget -nc https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.1/RealESRNet_x4plus.pth -P experiments/pretrained_models
    !wget -nc https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.1/ESRGAN_SRx4_DF2KOST_official-ff704c30.pth -P experiments/pretrained_models  #regular esrgan


    print("Finished Installing libraries for Real-ESRGAN upscaling.")
    %cd /content/
    #

#@markdown ##Git and pip install
!git clone https://github.com/openai/CLIP
!git clone https://github.com/crowsonkb/guided-diffusion
!pip install -e ./CLIP
!pip install -e ./guided-diffusion
!pip install lpips datetime
install_ESRGAN()

In [ ]:
#@markdown ##imports
import time
import gc
import io
import math
import sys
from IPython import display
import lpips
from PIL import Image, ImageOps
import requests
import torch
from torch import nn
from torch.nn import functional as F
import torchvision.transforms as T
import torchvision.transforms.functional as TF
from tqdm.notebook import tqdm
sys.path.append('./CLIP')
sys.path.append('./guided-diffusion')
import clip
from guided_diffusion.script_util import create_model_and_diffusion, model_and_diffusion_defaults
from datetime import datetime #filename
import numpy as np
import matplotlib.pyplot as plt
import random
import subprocess #future implementation
import os

In [ ]:
#@title  { form-width: "100px" }

# https://gist.github.com/adefossez/0646dbe9ed4005480a2407c62aac8869

def add_command(var,string):
  var = (var + string + ' ')

def image_resize(filepath,width):
  from PIL import Image
  basewidth = width
  img = Image.open(filepath)
  wpercent = (basewidth/float(img.size[0]))
  hsize = int((float(img.size[1])*float(wpercent)))
  #img = img.resize((basewidth,hsize), Image.ANTIALIAS)
  if width == 1024: img = img.resize((basewidth,hsize), Image.LANCZOS)
  else: img = img.resize((basewidth,hsize), Image.BICUBIC)
  img.save(filepath)


# def image_filter_edge(filepath):
#   #Import required image modules
#   from PIL import Image, ImageFilter
#   #Import all the enhancement filter from pillow
#   from PIL.ImageFilter import (
#     BLUR, CONTOUR, DETAIL, EDGE_ENHANCE, EDGE_ENHANCE_MORE,
#     EMBOSS, FIND_EDGES, SMOOTH, SMOOTH_MORE, SHARPEN
#   )
#   #Create image object
#   img = Image.open(filepath)
#   #Applying the blur filter
#   img1 = img.filter(EDGE_ENHANCE_MORE)
#   img1.save(filepath)
#   img1.show()

def interp(t):
    return 3 * t**2 - 2 * t ** 3

def perlin(width, height, scale=10, device=None):
    gx, gy = torch.randn(2, width + 1, height + 1, 1, 1, device=device)
    xs = torch.linspace(0, 1, scale + 1)[:-1, None].to(device)
    ys = torch.linspace(0, 1, scale + 1)[None, :-1].to(device)
    wx = 1 - interp(xs)
    wy = 1 - interp(ys)
    dots = 0
    dots += wx * wy * (gx[:-1, :-1] * xs + gy[:-1, :-1] * ys)
    dots += (1 - wx) * wy * (-gx[1:, :-1] * (1 - xs) + gy[1:, :-1] * ys)
    dots += wx * (1 - wy) * (gx[:-1, 1:] * xs - gy[:-1, 1:] * (1 - ys))
    dots += (1 - wx) * (1 - wy) * (-gx[1:, 1:] * (1 - xs) - gy[1:, 1:] * (1 - ys))
    return dots.permute(0, 2, 1, 3).contiguous().view(width * scale, height * scale)

def perlin_ms(octaves, width, height, grayscale, device=device):
    out_array = [0.5] if grayscale else [0.5, 0.5, 0.5]
    # out_array = [0.0] if grayscale else [0.0, 0.0, 0.0]
    for i in range(1 if grayscale else 3):
        scale = 2 ** len(octaves)
        oct_width = width
        oct_height = height
        for oct in octaves:
            p = perlin(oct_width, oct_height, scale, device)
            out_array[i] += p * oct
            scale //= 2
            oct_width *= 2
            oct_height *= 2
    return torch.cat(out_array)

def create_perlin_noise(octaves=[1, 1, 1, 1], width=2, height=2, grayscale=True):
    out = perlin_ms(octaves, width, height, grayscale)
    if grayscale:
        out = TF.resize(size=(side_x, side_y), img=out.unsqueeze(0))
        out = TF.to_pil_image(out.clamp(0, 1)).convert('RGB')
    else:
        out = out.reshape(-1, 3, out.shape[0]//3, out.shape[1])
        out = TF.resize(size=(side_x, side_y), img=out)
        out = TF.to_pil_image(out.clamp(0, 1).squeeze())

    out = ImageOps.autocontrast(out)
    return out

#################################################################################################################
def fetch(url_or_path):
    if str(url_or_path).startswith('http://') or str(url_or_path).startswith('https://'):
        r = requests.get(url_or_path)
        r.raise_for_status()
        fd = io.BytesIO()
        fd.write(r.content)
        fd.seek(0)
        return fd
    return open(url_or_path, 'rb')


def parse_prompt(prompt):
    if prompt.startswith('http://') or prompt.startswith('https://'):
        vals = prompt.rsplit(':', 2)
        vals = [vals[0] + ':' + vals[1], *vals[2:]]
    else:
        vals = prompt.rsplit(':', 1)
    vals = vals + ['', '1'][len(vals):]
    return vals[0], float(vals[1])

def sinc(x):
    return torch.where(x != 0, torch.sin(math.pi * x) / (math.pi * x), x.new_ones([]))

def lanczos(x, a):
    cond = torch.logical_and(-a < x, x < a)
    out = torch.where(cond, sinc(x) * sinc(x/a), x.new_zeros([]))
    return out / out.sum()

def ramp(ratio, width):
    n = math.ceil(width / ratio + 1)
    out = torch.empty([n])
    cur = 0
    for i in range(out.shape[0]):
        out[i] = cur
        cur += ratio
    return torch.cat([-out[1:].flip([0]), out])[1:-1]

def resample(input, size, align_corners=True):
    n, c, h, w = input.shape
    dh, dw = size

    input = input.reshape([n * c, 1, h, w])

    if dh < h:
        kernel_h = lanczos(ramp(dh / h, 2), 2).to(input.device, input.dtype)
        pad_h = (kernel_h.shape[0] - 1) // 2
        input = F.pad(input, (0, 0, pad_h, pad_h), 'reflect')
        input = F.conv2d(input, kernel_h[None, None, :, None])

    if dw < w:
        kernel_w = lanczos(ramp(dw / w, 2), 2).to(input.device, input.dtype)
        pad_w = (kernel_w.shape[0] - 1) // 2
        input = F.pad(input, (pad_w, pad_w, 0, 0), 'reflect')
        input = F.conv2d(input, kernel_w[None, None, None, :])

    input = input.reshape([n, c, h, w])
    return F.interpolate(input, size, mode='bicubic', align_corners=align_corners)

class MakeCutouts(nn.Module):
    def __init__(self, cut_size, cutn, skip_augs=False):
        super().__init__()
        self.cut_size = cut_size
        self.cutn = cutn
        self.skip_augs = skip_augs
        self.augs = T.Compose([
            T.RandomHorizontalFlip(p=0.5),
            T.Lambda(lambda x: x + torch.randn_like(x) * 0.01),
            T.RandomAffine(degrees=15, translate=(0.1, 0.1)),
            T.Lambda(lambda x: x + torch.randn_like(x) * 0.01),
            T.RandomPerspective(distortion_scale=0.4, p=0.7),
            T.Lambda(lambda x: x + torch.randn_like(x) * 0.01),
            T.RandomGrayscale(p=0.15),
            T.Lambda(lambda x: x + torch.randn_like(x) * 0.01),
            # T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
        ])

    def forward(self, input):
        input = T.Pad(input.shape[2]//4, fill=0)(input)
        sideY, sideX = input.shape[2:4]
        max_size = min(sideX, sideY)

        cutouts = []
        for ch in range(cutn):
            if ch > cutn - cutn//4:
                cutout = input.clone()
            else:
                size = int(max_size * torch.zeros(1,).normal_(mean=.8, std=.3).clip(float(self.cut_size/max_size), 1.))
                offsetx = torch.randint(0, abs(sideX - size + 1), ())
                offsety = torch.randint(0, abs(sideY - size + 1), ())
                cutout = input[:, :, offsety:offsety + size, offsetx:offsetx + size]

            if not self.skip_augs:
                cutout = self.augs(cutout)
            cutouts.append(resample(cutout, (self.cut_size, self.cut_size)))
            del cutout

        cutouts = torch.cat(cutouts, dim=0)
        return cutouts


def spherical_dist_loss(x, y):
    x = F.normalize(x, dim=-1)
    y = F.normalize(y, dim=-1)
    return (x - y).norm(dim=-1).div(2).arcsin().pow(2).mul(2)


def tv_loss(input):
    """L2 total variation loss, as in Mahendran et al."""
    input = F.pad(input, (0, 1, 0, 1), 'replicate')
    x_diff = input[..., :-1, 1:] - input[..., :-1, :-1]
    y_diff = input[..., 1:, :-1] - input[..., :-1, :-1]
    return (x_diff**2 + y_diff**2).mean([1, 2, 3])


def range_loss(input):
    return (input - input.clamp(-1, 1)).pow(2).mean([1, 2, 3])

##########################################################################################################

#@markdown ##def do_run()
def do_run():
  global firstRun,_easyScale
  loss_values = []

  if seed is not None:
      np.random.seed(seed)
      random.seed(seed)
      torch.manual_seed(seed)
      torch.cuda.manual_seed_all(seed)
      torch.backends.cudnn.deterministic = True

  make_cutouts = MakeCutouts(clip_size, cutn, skip_augs=skip_augs)
  target_embeds, weights = [], []

  for prompt in text_prompts:
      txt, weight = parse_prompt(prompt)
      txt = clip_model.encode_text(clip.tokenize(prompt).to(device)).float()
      #----
      print('This world could be signficantly less terrible\n')
      time.sleep(0.2)
      print('if we cared for one another just a little bit more.\n')
      time.sleep(0.4)
      print('Fork by sadnow aka @sadly_existent\n')
      time.sleep(0.2)
      print('Huge thanks to everyone involved! Especially the core founders of these tools.\n')
      time.sleep(0.2)
      print('Bugs can be reported through Github, Twitter, and VQLIPSE Discord...')
      time.sleep(0.5)
      if not _debug_mode:
            display.clear_output(wait=True)
      #----
      if fuzzy_prompt:
          for i in range(25):
              target_embeds.append((txt + torch.randn(txt.shape).cuda() * rand_mag).clamp(0,1))
              weights.append(weight)
      else:
          target_embeds.append(txt)
          weights.append(weight)

  for prompt in image_prompts:
      path, weight = parse_prompt(prompt)
      img = Image.open(fetch(path)).convert('RGB')
      img = TF.resize(img, min(side_x, side_y, *img.size), T.InterpolationMode.LANCZOS)
      batch = make_cutouts(TF.to_tensor(img).to(device).unsqueeze(0).mul(2).sub(1))
      embed = clip_model.encode_image(normalize(batch)).float()
      if fuzzy_prompt:
          for i in range(25):
              target_embeds.append((embed + torch.randn(embed.shape).cuda() * rand_mag).clamp(0,1))
              weights.extend([weight / cutn] * cutn)
      else:
          target_embeds.append(embed)
          weights.extend([weight / cutn] * cutn)

  target_embeds = torch.cat(target_embeds)
  weights = torch.tensor(weights, device=device)
  if weights.sum().abs() < 1e-3:
      raise RuntimeError('The weights must not sum to 0.')
  weights /= weights.sum().abs()

  init = None
  if init_image is not None:
      init = Image.open(fetch(init_image)).convert('RGB')
      init = init.resize((side_x, side_y), Image.LANCZOS)
      init = TF.to_tensor(init).to(device).unsqueeze(0).mul(2).sub(1)
  
  if perlin_init:
      if perlin_mode == 'color':
          init = create_perlin_noise([1.5**-i*0.5 for i in range(12)], 1, 1, False)
          init2 = create_perlin_noise([1.5**-i*0.5 for i in range(8)], 4, 4, False)
      elif perlin_mode == 'gray':
          init = create_perlin_noise([1.5**-i*0.5 for i in range(12)], 1, 1, True)
          init2 = create_perlin_noise([1.5**-i*0.5 for i in range(8)], 4, 4, True)
      else:
          init = create_perlin_noise([1.5**-i*0.5 for i in range(12)], 1, 1, False)
          init2 = create_perlin_noise([1.5**-i*0.5 for i in range(8)], 4, 4, True)
      
      # init = TF.to_tensor(init).add(TF.to_tensor(init2)).div(2).to(device)
      init = TF.to_tensor(init).add(TF.to_tensor(init2)).div(2).to(device).unsqueeze(0).mul(2).sub(1)
      del init2

  cur_t = None

  def cond_fn(x, t, y=None):
      with torch.enable_grad():
          x = x.detach().requires_grad_()
          n = x.shape[0]
          my_t = torch.ones([n], device=device, dtype=torch.long) * cur_t
          out = diffusion.p_mean_variance(model, x, my_t, clip_denoised=False, model_kwargs={'y': y})
          fac = diffusion.sqrt_one_minus_alphas_cumprod[cur_t]
          x_in = out['pred_xstart'] * fac + x * (1 - fac)
          x_in_grad = torch.zeros_like(x_in)
          for i in range(cutn_batches):
              clip_in = normalize(make_cutouts(x_in.add(1).div(2)))
              image_embeds = clip_model.encode_image(clip_in).float()
              dists = spherical_dist_loss(image_embeds.unsqueeze(1), target_embeds.unsqueeze(0))
              dists = dists.view([cutn, n, -1])
              losses = dists.mul(weights).sum(2).mean(0)
              loss_values.append(losses.sum().item()) # log loss, probably shouldn't do per cutn_batch
              x_in_grad += torch.autograd.grad(losses.sum() * clip_guidance_scale, x_in)[0] / cutn_batches
          tv_losses = tv_loss(x_in)
          range_losses = range_loss(out['pred_xstart'])
          sat_losses = torch.abs(x_in - x_in.clamp(min=-1,max=1)).mean()
          loss = tv_losses.sum() * tv_scale + range_losses.sum() * range_scale + sat_losses.sum() * sat_scale
          if init is not None and init_scale:
              init_losses = lpips_model(x_in, init)
              loss = loss + init_losses.sum() * init_scale
          x_in_grad += torch.autograd.grad(loss, x_in)[0]
          grad = -torch.autograd.grad(x_in, x, x_in_grad)[0]
      if clamp_grad:
          magnitude = grad.square().mean().sqrt()
          return grad * magnitude.clamp(max=0.05) / magnitude
      return grad

  if model_config['timestep_respacing'].startswith('ddim'):
      sample_fn = diffusion.ddim_sample_loop_progressive
  else:
      sample_fn = diffusion.p_sample_loop_progressive

  for i in range(n_batches):
      cur_t = diffusion.num_timesteps - skip_timesteps - 1

      if model_config['timestep_respacing'].startswith('ddim'):
          samples = sample_fn(
              model,
              (batch_size, 3, side_y, side_x),
              clip_denoised=clip_denoised,
              model_kwargs={},
              cond_fn=cond_fn,
              progress=True,
              skip_timesteps=skip_timesteps,
              init_image=init,
              randomize_class=randomize_class,
              eta=eta,
          )
      else:
          samples = sample_fn(
              model,
              (batch_size, 3, side_y, side_x),
              clip_denoised=clip_denoised,
              model_kwargs={},
              cond_fn=cond_fn,
              progress=True,
              skip_timesteps=skip_timesteps,
              init_image=init,
              randomize_class=randomize_class,
          )

      for j, sample in enumerate(samples):
          if not _debug_mode:
            display.clear_output(wait=True)
          cur_t -= 1
          if j % display_rate == 0 or cur_t == -1:  #Only single iteration has finished
              for k, image in enumerate(sample['pred_xstart']):
                  #################################################################
                  tqdm.write(f'Batch {i}, step {j}, output {k}:')
                  if firstRun: max_mutation_amount = 0
                  tqdm.write(f'Max mutation {max_mutation_amount}, Guidance scale {clip_guidance_scale}, TV scale {tv_scale}, Range scale {range_scale}')
                  tqdm.write(msg_runtime)
                  #################################################################
                  current_time = datetime.now().strftime('%y%m%d-%H%M%S_%f')
                  #filename = f'progress_batch{i:05}_iteration{j:05}_output{k:05}_{current_time}.png'
                  filename = f'batch{i:03}_iteration{j:03}_output{k:03}_{current_time}.png'  #reduced padding
                  image = TF.to_pil_image(image.add(1).div(2).clamp(0, 1))
                  image.save('/content/image_storage/' + filename)
                  display.display(display.Image('/content/image_storage/' + filename))
                  if _batch_genetics or _init_genetics:
                    if firstRun:
                      print(firstRun)  #debug
                      max_mutation_amount = (_max_init_variance) / (n_batches)
                      #print("Max mutation per output is ",max_mutation_amount)  #debug
                      firstRun = False #we don't want this calculating again until next run
                    #import random
                    actual_mutation_amount = random.uniform(0,max_mutation_amount)
                    _easyScale = _easyScale + actual_mutation_amount
                    calculate_easyScale()
                    #cond_fn()
                    cond_fn=cond_fn #unsure whether thse will work

                  if google_drive and cur_t == -1:  #IMAGE HAS FINISHED & DRIVE IS ENABLED
                    gc.collect()
                    torch.cuda.empty_cache()
                    #Image_upscale(filename)  #unsued resizer function
                    #INSERT CODE ON THIS LINE
                    if not _skip_upscaling: Image_upscale(filename,image)  #brought to you by world hunger  
                    else: image.save(output_folder_images + filename)
                  #else:                   
                    #anything placed here will run before the upscale has completed
                    #will be ran every single step
                    #image.save(output_folder_images + filename)

                      

      plt.plot(np.array(loss_values), 'r')


# Settings & Generation

In [ ]:
timestep_respacing = 'ddim50' #@param ['25','50','150','250','500','1000','ddim25','ddim50','ddim150','ddim250','ddim500','ddim1000']
#@markdown *Modify this value to decrease the number of iterations/prompt.
# timestep_respacing = '25'
diffusion_steps = 1000

model_config = model_and_diffusion_defaults()
if diffusion_model == '512x512_diffusion_uncond_finetune_008100':
    model_config.update({
        'attention_resolutions': '32, 16, 8',
        'class_cond': False,
        'diffusion_steps': diffusion_steps,
        'rescale_timesteps': True,
        'timestep_respacing': timestep_respacing,
        'image_size': 512,
        'learn_sigma': True,
        'noise_schedule': 'linear',
        'num_channels': 256,
        'num_head_channels': 64,
        'num_res_blocks': 2,
        'resblock_updown': True,
        'use_fp16': True,
        'use_scale_shift_norm': True,
    })
elif diffusion_model == '256x256_diffusion_uncond':
    model_config.update({
        'attention_resolutions': '32, 16, 8',
        'class_cond': False,
        'diffusion_steps': diffusion_steps,
        'rescale_timesteps': True,
        'timestep_respacing': timestep_respacing,
        'image_size': 256,
        'learn_sigma': True,
        'noise_schedule': 'linear',
        'num_channels': 256,
        'num_head_channels': 64,
        'num_res_blocks': 2,
        'resblock_updown': True,
        'use_fp16': True,
        'use_scale_shift_norm': True,
    })
side_x = side_y = model_config['image_size']

model, diffusion = create_model_and_diffusion(**model_config)
model.load_state_dict(torch.load(f'{model_path}{diffusion_model}.pt', map_location='cpu'))
model.requires_grad_(False).eval().to(device)
for name, param in model.named_parameters():
    if 'qkv' in name or 'norm' in name or 'proj' in name:
        param.requires_grad_()
if model_config['use_fp16']:
    model.convert_to_fp16()

################################################

clip_model = clip.load('ViT-B/16', jit=False)[0].eval().requires_grad_(False).to(device)
clip_size = clip_model.visual.input_resolution
normalize = T.Normalize(mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])
lpips_model = lpips.LPIPS(net='vgg').to(device)

In [ ]:
#file
  #open
#edit
#view
########################################################################
#@title  { form-width: "300px" }
#@title  { form-width: "50px" }
firstRun = True
msg_runtime = ''
_keep_first_upscale = False
#############################################################################################
#@markdown ##Simple Settings
#@markdown ---
_text_prompt =  "Photorealistic Halloween decorations,'Halloween Night' made in unreal engine,spooky Halloween jack-o-lanterns" #@param {type:"string"} 
  # This will be the name of your project folder in Drive.
_project_name = 'new360-tests' #@param{type:"string"}
_easyScale = 1 #@param {type:"slider", min:0, max:2, step:0.1}

#@markdown ---
_run_upscaler = True
text_prompts = [
    # "an abstract painting of 'ravioli on a plate'",
    # 'cyberpunk wizard on top of a skyscraper, trending on artstation, photorealistic depiction of a cyberpunk wizard',
    _text_prompt]
    # 'cyberpunk wizard',


n_batches =  1000#@param{type:"raw"}
batch_size =  1#@param{type:"raw"}
_noise_mode = 'gray' #@param ['mixed','gray','color']
_noise_amount = 0.05 #@param {type:"slider", min:0.00, max:1, step:0.01}


#--------------------------------------------------------------------------------------------------------
#@markdown ---
#_upscale_multiplier = "8" #@param [4,8]



#@markdown ---
#--------------------------------------------------------------------------------------------------------


#@markdown ---

#@markdown ##Init Settings (Optional)
_init_image = '' #@param{type:"string"}
if _init_image == '':
  init_image = None
else:
  init_image = _init_image  # None - URL or local path
  msg_runtime = msg_runtime + 'Init image: ' + init_image + ' \n'

####Noise & Init Setting Overrides #--------------------------------------------------------------------------
if _noise_amount > 0: 
  add_random_noise = True
else:
  add_random_noise = False

if not _init_image == '':  #to prevent noise from messing with init
  if add_random_noise == True:
    msg_runtime = msg_runtime + 'Notice: init_images have mixed results when _noise_amount > 0 \n'
  # _noise_amount = 0
  # add_random_noise = False

perlin_init = add_random_noise
rand_mag = _noise_amount # 0.1 - Controls the magnitude of the random noise
_init_scale =  0#@param{type:"raw"}
#default_init_scale = 1000 #in case user forgets to set
if _init_image == '':
  print("No init image detected. Setting init_scale to 0...")
  init_scale = 0
else:
  init_scale = _init_scale
  if init_scale == 0:
    msg_runtime = msg_runtime + 'Notice: init_image is set but there is no init_scale! Try 1000 as a default. \n'
    #init_scale = default_init_scale
#--------------------------------------------------------------------------------------------------
_image_prompts = "" #@param{type:"string"}
if not _image_prompts == "":
  msg_runtime = msg_runtime + 'Image prompt: ' + _image_prompts + ' \n'
  image_prompts = [
      _image_prompts,
  ]
else: image_prompts = []

skip_timesteps =  7#@param{type:"raw"}

#@markdown #Genetics (Experimental, Untested)

_batch_genetics = False #@param{type:"boolean"}
_init_genetics = False  #not currently implemented
_max_init_variance =  0.1#@param{type:"raw"}

# @markdown I have no idea if `genetics` is actually being applied. Just that the vars are being changed. If you figure it out let me know?



##@markdown ---
#@markdown ##Advanced Settings (Optional)

# 350/50/50/32 and 500/0/0/64 have worked well for 25 timesteps on 256px
cutn_batches = 1 #@param [1,2,4,8,16] {type:"raw"}
cutn =  96#@param{type:"raw"}
perlin_mode = _noise_mode # 'mixed' ('gray', 'color')

# 2 - cutn_batch Accumulate CLIP gradient from multiple batches of cuts [Can help with OOM errors / Low VRAM]





#_enhance_upscale = True #@param{type:"boolean"}

  # 0 - Controls the starting point along the diffusion timesteps
_saturation_scale =  0#@param{type:"raw"}
sat_scale = _saturation_scale
  # 0 - Controls how much saturation is allowed. From nshepperd's JAX notebook.

#--------------------------------------------------------------------------------------------
#@markdown ---
_clip_guidance_scale =  4000#@param {type:"raw"}
_tv_scale =  150#@param {type:"raw"}
_range_scale =  150#@param {type:"raw"}
#@markdown ---
#--------------------------------------------------------------------------------------------
##@markdown `skip_timesteps` best 5 (thx steven), 10 for dd50
skip_augs = False # False - Controls whether to skip torchvision augmentations
randomize_class = True # True - Controls whether the imagenet class is randomly changed each iteration
clip_denoised = True #@param{type:"boolean"}
##@markdown False - Determines whether CLIP discriminates a noisy or denoised image
clamp_grad = True # True - Experimental: Using adaptive clip grad in the cond_fn
seed = None
# seed = random.randint(0, 2**32) # Choose a random seed and print it at end of run for reproduction
# if _diffusion_int == 512: display_rate = 2
# else: display_rate = 1
display_rate = 1
fuzzy_prompt = False #@param{type:"boolean"}
##@markdown False - Controls whether to add multiple noisy prompts to the prompt losses
eta = 0.5 #@param{type:"raw"}
##@markdown 0.0 - DDIM hyperparameter

global _debug_mode
_debug_mode = False #@param{type:"boolean"}
#@markdown ---


################################################
#---------------------------------------------------------------------------------------
#Output image directory handling
import os
output_folder_images = os.path.join(_drive_location,_project_name)
# contains_slash = (output_folder_images.endswith('/'))
# if not contains_slash:
#   output_folder_images = output_folder_images + '/'
dir_make(output_folder_images)
temp_image_storage = '/content/image_storage/'  #for non-upscaled images
dir_make(temp_image_storage)
#---------------------------------------------------------------------------------------
def calculate_easyScale():
  global clip_guidance_scale,tv_scale,range_scale
  clip_guidance_scale = _easyScale * _clip_guidance_scale
  tv_scale = _easyScale * _tv_scale
  range_scale = _easyScale * _range_scale
  return clip_guidance_scale,tv_scale,range_scale
calculate_easyScale()

#---------------------------------
# UPSCALING
_skip_upscaling = True #@param{type:"boolean"}
_upscale_model='RealESRGAN 4x' #@param ['(Regular) ESRGAN 4x','RealESR_NET 4x','RealESRGAN 2x','RealESRGAN 4x','RealESRGAN x4 Anime_6B']

msg_runtime = msg_runtime + 'Upscaler model: ' + _upscale_model + ' \n'
if _upscale_model == '(Regular) ESRGAN 4x': _upscale_model = 'ESRGAN_SRx4_DF2KOST_official-ff704c30.pth'
if _upscale_model == 'RealESR_NET 4x': _upscale_model = 'RealESRNet_x4plus.pth'
if _upscale_model == 'RealESRGAN 2x': _upscale_model = 'RealESRGAN_x2plus.pth'
if _upscale_model == 'RealESRGAN 4x': _upscale_model = 'RealESRGAN_x4plus.pth'
if _upscale_model == 'RealESRGAN x4 Anime_6B': _upscale_model = 'RealESRGAN_x4plus_anime_6B.pth'
upscale_model_fullpath='/content/Real-ESRGAN/experiments/pretrained_models/' + _upscale_model
if upscale_model_fullpath == '/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x2plus.pth': upscale_value="2"
#else: upscale_value="4"


_esrgan_tilesize = "512" #@param[16,32,64,128,256,512,1024]
#_upscale_performance_mode = False
_outscale_resolution = "2048" #@param[256,512,1024,2048,4096]



def Image_upscale(filename,image):   
  def round_up_to_even(f):
    return math.ceil(f / 2.) * 2

  def File_addSuffix(filename,needed_passes,completed_passes):
    #returns the filename of the previously upscaled file
    #if not completed_passes == needed_passes:
    stripped_filename = os.path.splitext(filename)[0] #get non-upscaled image filename
    stripped_filename = os.path.join(_project_name,stripped_filename)
    z = 0
    while z < completed_passes:
      stripped_filename = stripped_filename + '_out'
      z = z + 1
    if not (completed_passes + 1) == needed_passes:
      target_extension = 'png'  #dont forget the dot
    else: target_extension = 'jpg'
    stripped_filename = stripped_filename + '.' + target_extension  #this identifies the fullpath output from the 1st upscale
    if completed_passes > 0:
      current_upscale_target = os.path.join(output_folder_images,stripped_filename)
      #current_upscale_target = os.path.join(temp_image_filepath,stripped_filename)
    #else: output_extension = 'png'
    print('Current upscale target is ',current_upscale_target)
    return current_upscale_target
  
  %cd /content/Real-ESRGAN/
  temp_image_filepath = temp_image_storage + filename
  #image.save(temp_image_filepath) #here we save the un-upscaled so we can load it IN the upscaler
  
  retry_attempt = 0

  
  def Load_config(retry_attempt,_upscale_performance_mode):
    needed_passes = 1
    global _skip_upscaling
    if retry_attempt > 1:
      print("UPSCALING FAILED! Try raising your cutn_batch settings.")
      print("Attempting to save un-upscaled file...")
      image.save(output_folder_images + filename)
      _skip_upscaling = True
      import sys
      sys.exit()

    if diffusion_model == "512x512_diffusion_uncond_finetune_008100": model_size = 512
    else: model_size = 256
    global _outscale_resolution
    outscale_resolution_int = int(_outscale_resolution)
    _outscale = outscale_resolution_int / model_size
      #ex: 2048 / 512 = _outscale of 4
    if not (_outscale % 2) == 0: #number is not even
      _outscale = round_up_to_even(_outscale)
      #if _outscale > 4: _outscale = 4
    if _upscale_performance_mode:
      _outscale = _outscale / 2
      needed_passes = needed_passes * 2
      half_precision = True
    if _outscale > 4 :
      needed_passes = _outscale / 4
    else:
      # _outscale = 4
      # needed_passes = 2
      _skip_upscaling = False
      half_precision = False
    needed_passes = round(needed_passes)
    print("NOTICE: Performing ",needed_passes,"needed_passes...")
    print("Outscale is", _outscale)
    print("Needed passes is", needed_passes)
    return _outscale,needed_passes,_skip_upscaling,half_precision

  run_config = True
  #double_pass = False
  #second_pass = False
  _upscale_performance_mode = False

  completed_passes = 0

  retry_attempt = 0
  upscale_complete = False
  
  while not upscale_complete:
    if run_config:
      _outscale,needed_passes,_skip_upscaling,half_precision = Load_config(retry_attempt,_upscale_performance_mode)
      completed_passes = 0
    end_flag = '--face_enhance '
    #--input $current_upscale_target
    _outscale = str(_outscale)
    if half_precision: end_flag = end_flag + '--half '
    if not _upscale_model == 'RealESRGAN 2x': end_flag = end_flag + '--outscale ' + _outscale + ' '

    #current_upscale_target = File_addSuffix(filename,needed_passes,completed_passes)
    if completed_passes < 1:
        end_flag = end_flag + '--input ' + temp_image_filepath + ' '
    else:
      current_upscale_target = File_addSuffix(filename,needed_passes,completed_passes)
      end_flag = end_flag + '--input ' + current_upscale_target + ' '
    if end_flag.endswith(' '): end_flag = end_flag[:-1]

    if completed_passes > 0 or needed_passes == 1:
      output_extension = 'jpg'
    else: output_extension = 'png'
    
    if not upscale_complete:
      !python inference_realesrgan.py --model_path $upscale_model_fullpath --output $output_folder_images --tile $_esrgan_tilesize --ext $output_extension $end_flag
    
    # if double_pass: completed_passes = completed_passes + 1
    # else: completed_passes = completed_passes + 2
    if completed_passes > 0: _upscaled_path = file_check(current_upscale_target)
    else: _upscaled_path = file_check(temp_image_filepath)  #attempting to load in performance_mode
    if _upscaled_path:
      print("NOTICE: Upscale pass ",completed_passes," COMPLETE!")
      #if double_pass: second_pass = True
      run_config = False  #because we are done
      completed_passes = completed_passes + 1

    if _upscaled_path:  
        !rm $current_upscale_target
        #return        
    else: 
      print("ERROR: UPSCALE DID NOT COMPLETE!")
      if not _upscale_performance_mode:
        print("RECOVERY STATUS: ATTEMPTING TO RUN IN upscale_performance mode...")
        _upscale_performance_mode = True
      completed_passes = 0
      retry_attempt = retry_attempt + 1
      run_config = True
    if completed_passes + 1 >= needed_passes:
      upscale_complete = True
    if upscale_complete:
      return

if _debug_mode:
  print("Setting skip to large number,,,")
  skip_timesteps = 23

  # !python inference_realesrgan.py --model_path $double_upscale_model_fullpath --input $current_upscale_target --output $output_folder_images --tile $_esrgan_tilesize --ext jpg --face_enhance 
    
  #   !python inference_realesrgan.py --model_path $upscale_model_fullpath --face_enhance --input $temp_image_filepath --output $output_folder_images --ext $output_extension --tile $_esrgan_tilesize --outscale $_outscale_resolution --half
  #   #image_resize(temp_image_filepath,512) #------------------
  
  # if upscale_multiplier > 4 or _diffusion_int == 512:  #double-upscaling
  #   #print("Performing the second upscale at half precision...")
  #   gc.collect()
  #   import os

  #   #current_upscale_target = output_folder_images + stripped_filename
  #   # if _enhance_upscale:
  #   #   print("Downscaling first upscaled img...")
  #   #   image_resize(current_upscale_target,512)
  #   print("Attempting to upscale ",current_upscale_target)
  #   %cd /content/Real-ESRGAN/
  #   #if (_double_upscale_model == 'RealESRGAN 2x') or (_upscale_model == 'RealESRGAN 2x'):
  #   if _diffusion_int == 512 or upscale_multiplier > 2:
  #     !python inference_realesrgan.py --model_path $double_upscale_model_fullpath --face_enhance --input $current_upscale_target --output $output_folder_images --ext jpg --tile $_esrgan_tilesize --outscale $_outscale_resolution --half
  #   else:
  #     !python inference_realesrgan.py --model_path $double_upscale_model_fullpath --face_enhance --input $current_upscale_target --output $output_folder_images --ext jpg --tile $_esrgan_tilesize --outscale $_outscale_resolution 

    #subprocess.call(upscale_command,cwd='/content/Real-ESRGAN/',capture_output=True,shell=True)

      #so that only the final upscale remains
    #time.sleep(10)



# _double_upscale_model='RealESRGAN x4 Anime_6B' #@param ['(Regular) ESRGAN 4x','RealESR_NET 4x','RealESRGAN 2x','RealESRGAN 4x','RealESRGAN x4 Anime_6B']
# msg_runtime = msg_runtime + 'Double-upscale model: ' + _double_upscale_model + ' \n'
# if _double_upscale_model == '(Regular) ESRGAN 4x': double_upscale_model = 'ESRGAN_SRx4_DF2KOST_official-ff704c30.pth'
# if _double_upscale_model == 'RealESR_NET 4x': double_upscale_model = 'RealESRNet_x4plus.pth'
# if _double_upscale_model == 'RealESRGAN 2x': double_upscale_model = 'RealESRGAN_x2plus.pth'
# if _double_upscale_model == 'RealESRGAN 4x': double_upscale_model = 'RealESRGAN_x4plus.pth'
# if _double_upscale_model == 'RealESRGAN x4 Anime_6B': double_upscale_model = 'RealESRGAN_x4plus_anime_6B.pth'
# double_upscale_model_fullpath='/content/Real-ESRGAN/experiments/pretrained_models/' + double_upscale_model
# double_upscale_value="2"
# '/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x4plus.pth'
#-------------------------------------------------------
#@markdown Original Defaults: `clip_guidance_scale 5000`,`tv_scale 150`,`range scale 150`

#@markdown Recommended defaults for init_images (ddim50): `clip_guidance_scale 2000`,`tv_scale 150`,`range scale 50`, `init_scale 1000`, `skip_timesteps 16 (7-9 for ddim25)`

#@markdown There is a possibility that `tv_scale` can be set between `0` to `10000`

#@markdown `skip_timesteps` does a lot for the similarity in `init_settings`

#@markdown Special thanks to many people on the VQLIPSE Discord




# 1 - Controls how many consecutive batches of images are generated

gc.collect()
torch.cuda.empty_cache()
try:
    do_run()
except KeyboardInterrupt:
    gc.collect()
    torch.cuda.empty_cache()
    pass
finally:
    print('seed', seed)
    print('Output(s) saved to ',output_folder_images)
    gc.collect()
    torch.cuda.empty_cache()

###############################################

# Prompt Engineering (Documentation)

Collapse this section if it makes you feel cramped-

`_easyScale` controls a few of the below advanced settings. Experiment!

`_init_image` blank if you want to generate from scratch.
`cutn_batches` might increase quality at the cost of speed. Lowering it may help with VRAM issues.
A possible good value for `_init_scale` is 1000. If set to `0` it will default back to 1000 when an init image is present.

`_noise` settings are deactivated if an `init_image` is detected.

Set `saturation_scale` to `0` to use native saturation control (default). 



**Examples of diffusion prompts:**

- `Representation of chronic anxiety:2, illustrated by Luigi Serafini:1, inspired by the Codex Seraphinianus:1`

- `Sad panda vector art made in Blender 3d:4, nature background:2, 4k parallax vibrant colorful panda bear trending on artstation:1` with init scale of 5, init image of https://www.pngitem.com/pimgs/m/16-161339_giant-panda-bear-silhouette-drawing-clip-art-silhouette.png

- `Representation of depression | isometric vector art titled "Depression"` (Yes I accidently used double quotes but still looked good)

- `Liminal space, liminal hotel hallway rendered in unreal engine, top post on r/liminalspaces`

*If you use `:` (weights), you must weigh out everything or you'll receive an error. For example, if you have three things and one has a weight of `:2`, the other two things need to have a weight of `:1`.

*I am currently unsure whether `|` (pipes) do the same thing as commas.*

---

Diffusion prompting might seem trickier to master than VQGAN models, but it still allows for some level of control. Sometimes less it better.

It doesn't always need the `trending on artstation` type of lingo; sometimes it will benefit more from something like `photorealistic 4k nature replication`.

Diffusion is kind of unique in how it often doesn't duplicate the desired subject. For example, `cat photo` will usually only give you 1 cat.

------------------------------------------------

### A Giant List of Terms to Try

Credit for this list goes to @Atman on Discord and/or whoever else contributed to the Pastebin.

*'8k resolution'
,'pencil sketch'
,'8K 3D'
,'creative commons attribution'
,'deviantart'
,'CryEngine'
,'Unreal Engine'
,'concept art'
,'photoillustration'
,'pixiv'
,'Flickr'
,'ArtStation HD'
,'Behance HD'
,'HDR'
,'anime'
,'filmic'
,'Stock photo'
,'Ambient occlusion'
,'Global illumination'
,'Chalk art'
,'Low poly'
,'Booru'
,'Polycount'
,'Acrylic art'
,'Hyperrealism'
,'Zbrush Central'
,'Rendered in Cinema4D'
,'Rendered in Maya'
,'Photo taken with Nikon D750'
,'Tilt shift'
,'Mixed media'
,'Depth of field'
,'DSLR'
,'Detailed painting'
,'Volumetric lighting'
,'Storybook illustration'
,'Unsplash contest winner'
,'#vfxfriday'
,'Ultrafine detail'
,'20 megapixels'
,'Photo taken with Fujifilm Superia'
,'Photo taken with Ektachrome'
,'matte painting'
,'reimagined by industrial light and magic'
,'Watercolor'
,'CGSociety'
,'childs drawing'
,'marble sculpture'
,'airbrush art'
,'renaissance painting'
,'Velvia'
,'Provia'
,'photo taken with Provia'
,'prerendered graphics'
,'criterion collection'
,'dye-transfer'
,'stipple'
,'Parallax'
,'Bryce 3D'
,'Terragen'
,'(2013) directed by cinematography by'
,'Bokeh'
,'1990s 1995'
,'1970s 1975'
,'1920s 1925'
,'charcoal drawing'
,'commission for'
,'furaffinity'
,'flat shading'
,'ink drawing'
,'artwork'
,'oil on canvas'
,'macro photography'
,'hall of mirrors'
,'polished'
,'sunrays shine upon it'
,'aftereffects'
,'iridescent'
,'#film'
,'datamosh'
,'(1962) directed by cinematography'
,'holographic'
,'dutch golden age'
,'digitally enhanced'
,'National Geographic photo'
,'Associated Press photo'
,'matte background'
,'Art on Instagram'
,'#myportfolio'
,'digital illustration'
,'stock photo'
,'aftereffects'
,'speedpainting'
,'colorized'
,'detailed'
,'psychedelic'
,'wavy'
,'groovy'
,'movie poster'
,'pop art'
,'made of beads and yarn'
,'made of feathers'
,'made of crystals'
,'made of liquid metal'
,'made of glass'
,'made of cardboard'
,'made of vines'
,'made of cheese'
,'made of flowers'
,'made of insects'
,'made of mist'
,'made of paperclips'
,'made of rubber'
,'made of plastic'
,'made of wire'
,'made of trash'
,'made of wrought iron'
,'made of all of the above'
,'tattoo'
,'woodcut'
,'American propaganda'
,'Soviet propaganda'
,'PS1 graphics'
,'Fine art'
,'HD mod'
,'Photorealistic'
,'Poster art'
,'Constructivism'
,'pre-Raphaelite'
,'Impressionism'
,'Lowbrow'
,'RTX on'
,'chiaroscuro'
,'Egyptian art'
,'Fauvism'
,'shot on 70mm'
,'Art Deco'
,'Picasso'
,'Da Vinci'
,'Academic art'
,'3840x2160'
,'Photocollage'
,'Cubism'
,'Surrealist'
,'THX Sound'
,'ZBrush'
,'Panorama'
,'smooth'
,'DC Comics'
,'Marvel Comics'
,'Ukiyo-e'
,'Flemish Baroque'
,'vray tracing'
,'pixel perfect'
,'quantum wavetracing'
,'Zbrush central contest winner'
,'ISO 200'
,'Bob Ross'
,'32k HUHD'
,'Photocopy'
,'DeviantArt HD'
,'infrared'
,'Angelic photograph'
,'Demonic photograph'
,'Biomorphic'
,'Windows Vista'
,'Skeuomorphic'
,'Physically based rendering'
,'Trance compilation CD'
,'Concert poster'
,'Steampunk'
,'Sketchfab'
,'Goth'
,'Wiccan'
,'trending on artstation'
,'featured on artstation'
,'artstation HQ'
,'artstation contest winner'
,'ultra HD'
,'high quality photo'
,'instax'
,'ilford HP5'
,'infrared'
,'Lomo'
,'Matte drawing'
,'matte photo'
,'glowing neon'
,'Xbox 360 graphics'
,'flickering light'
,'Playstation 5 screenshot'
,'Kodak Gold 200'
,'by Edward Hopper'
,'rough'
,'maximalist'
,'minimalist'
,'Kodak Ektar'
,'Kodak Portra'
,'geometric'
,'cluttered'
,'Rococo'
,'destructive'
,'by James Gurney'
,'by Thomas Kinkade'
,'by Vincent Di Fate'
,'by Jim Burns'
,'androgynous'
,'masculine'
,'genderless'
,'feminine'
,'extremely gendered, masculine and feminine'
,'4k result'
,'#pixelart'
,'voxel art'
,'wimmelbilder'
,'dystopian art'
,'apocalypse art'
,'apocalypse landscape'
,'2D game art'
,'Windows XP'
,'y2k aesthetic'
,'#screenshotsaturday'
,'seapunk'
,'vaporwave'
,'Ilya Kuvshinov'
,'Paul Cezanne'
,'Henry Moore'
,'phallic'
,'creepypasta'
,'retrowave'
,'synthwave'
,'outrun'*